In [43]:
import requests

In [62]:
SPOTIFY_GET_CURRENT_USER_PROFILE_URL = "https://api.spotify.com/v1/me/player/recently-played?limit=3"
ACCESS_TOKEN = ''

In [94]:
def get_user_id(access_token):
    response = requests.get(
        SPOTIFY_GET_USER_PROFILE_URL,
        headers = {
            "Content-type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
    )
    json_response = response.json()
    return json_response

def get_current_user_recent_tracks(access_token):
    response = requests.get(
        SPOTIFY_GET_CURRENT_USER_PROFILE_URL,
        headers = {
            "Content-type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
    )
    json_response = response.json()
    return json_response


ACCESS_TOKEN = "BQAF1zfmD54F5rY4u0ZiKDGZWTS5ud5dQgGSJqb66DAgAJ6xlr2VQo4sbHQT8-b6ubkPkbZqhUJArxpXIMR44xCDGGjL_1WJXCEeFkEfBwcBVXx0UJc913zkA7ACC5Kpw-Qp6U6j80-S3WBgsnjBxsogIt1W67e4heEi-mDkMZCzqwpl8MkPlgRi"
    #json_resp = get_user_id(ACCESS_TOKEN)
json_resp = get_current_user_recent_tracks(ACCESS_TOKEN)
for track in json_resp['items']:
    print(track['track']['id'])
    print(track['track']['name'])
    print(track['track']['artists'][0]['name'])

1N9M3x7kheonbmXAh6CS7d
Red Sippy Cup
Ellis G
6wVMFTx9JyUDqPYzokDEpw
Rockstar
Abhi The Nomad
0GPJSHYaXh8rZSSJoUMgyl
free love
HONNE
